In [ ]:
import math

def gcd(a, b):
    if b == 0:
        return a  # Base case: GCD is the remaining non-zero number
    else:
        return gcd(b, a % b)

In [ ]:

def pollard_rho(n):
    if n % 2 == 0:
        return 2

    x = 2  # Starting value
    y = 2  # Second starting value
    d = 1  # Placeholder for potential factor

    while d == 1:
        x = pow(x, 2, n) + 1  # Modified function for better cycle detection
        y = pow(y, 2, n) + 1
        y = pow(y, 2, n) + 1
        d = gcd(abs(x - y), n)

    if d == n:
        return None  # No non-trivial factor found
    else:
        return d

# Example usage


In [ ]:
number_to_factor = 104034233
factor = pollard_rho(number_to_factor)
if factor:
    print("Factor found:", factor)
else:
    print("No factor found within a reasonable time.")


Factor found: 31


In [ ]:
number_to_factor/31

3355943.0